- (Gaussian) Multinomial Naive Bayes with Grid Search
- Logistic Regression
- Support Vector Machines
- Decision Tree
- Random Forest & Bagging Classifier
- Boosting XGBoost
- Neural Nets



In [41]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as datasets
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Load Dataset

In [3]:
file = open("../data/processed/data.p",'rb')
data = pickle.load(file)
file.close()

file = open("../data/processed/labels.p",'rb')
labels = pickle.load(file)
file.close()

In [4]:
label = labels[:,0]
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))

embeddings: (1450, 200)
label: (1450,)


In [5]:
# get rid of Nones
# for i in range(label.shape[0]):
#     print(label[i])
#     if label[i] == None:
#         print(i)
# 299 -> 448

In [6]:
label = np.concatenate((label[:298], label[449::]), 0)
label=label.astype('int')
data = np.concatenate((data[:298], data[449::]), 0)
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))

embeddings: (1299, 200)
label: (1299,)


### Support Vector Machine: Multi-Class Classification

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42, stratify=label)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1039, 200) (260, 200) (1039,) (260,)


In [8]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

### Test on Training Data

In [9]:
pred_train = clf.predict(X_train)
accuracy_score(y_train, pred_train)

0.7247353224254091

In [10]:
confusion_matrix(y_train, pred_train)

array([[ 20,  13,   0,   5,   1,   0,   0,   2,   0,   0,  13],
       [  1,  95,   0,   4,   1,   0,   0,   0,   0,   0,  12],
       [  2,  13,   2,   1,   0,   0,   0,   0,   0,   0,   5],
       [  2,  19,   0,  41,   1,   0,   0,   0,   0,   0,  23],
       [  1,  11,   0,   2,  20,   0,   0,   0,   0,   1,   9],
       [  2,  12,   0,   5,   0,   3,   0,   0,   0,   0,  14],
       [  0,   4,   0,   2,   0,   0,   6,   1,   0,   1,   4],
       [  2,   9,   0,   9,   1,   0,   0,   4,   0,   1,  21],
       [  0,   5,   0,   1,   0,   0,   0,   0,   0,   0,   0],
       [  3,  18,   0,   5,   0,   0,   0,   0,   0,   9,  17],
       [  1,   3,   0,   3,   0,   0,   0,   0,   0,   0, 553]],
      dtype=int64)

### Test on Testing Data

In [11]:
pred_test = clf.predict(X_test)
accuracy_score(y_test, pred_test)

0.6538461538461539

In [12]:
confusion_matrix(y_test, pred_test)

array([[  6,   4,   0,   1,   0,   0,   0,   0,   0,   1,   1],
       [  0,  19,   0,   2,   0,   0,   0,   0,   0,   0,   7],
       [  0,   2,   0,   0,   2,   0,   0,   0,   0,   0,   2],
       [  0,   8,   0,   4,   1,   0,   0,   0,   0,   0,   9],
       [  0,   3,   0,   1,   5,   0,   0,   0,   0,   0,   2],
       [  0,   2,   0,   1,   1,   0,   0,   0,   0,   0,   5],
       [  1,   1,   1,   0,   0,   0,   0,   0,   0,   0,   1],
       [  0,   7,   0,   0,   0,   0,   0,   1,   0,   0,   4],
       [  0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   1],
       [  0,   3,   0,   1,   1,   0,   0,   0,   0,   0,   8],
       [  0,   3,   0,   2,   0,   0,   0,   0,   0,   0, 135]],
      dtype=int64)

## Linear Regression & Evaluation of Different Types of Regularization

## Vanilla Linear Regression (No Regularization)

In [86]:
reg = LinearRegression().fit(X_train, y_train)
print("Training Accuracy: " + str(reg.score(X_train, y_train)))
print("Training Accuracy: " + str(reg.score(X_test, y_test)))

Training Accuracy: 0.6195824242931637
Training Accuracy: 0.4464623202341178


## Ridge Regression (L2 Regularization)

In [98]:
parameters = {'alpha':[1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10]}
rid = Ridge()
clf = GridSearchCV(rid, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Training Accuracy: 0.5454773677585926
Testing Accuracy: 0.5169477873199229
{'alpha': 2}


## Lasso Regression (L1 Regularization)

In [99]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100]}

las = Lasso(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Training Accuracy: 0.5869160640569879
Testing Accuracy: 0.49856041968169207
{'alpha': 0.001}


## ElasticNet Regression (L1 + L2 Regularization)

In [103]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100], 'l1_ratio':[0.05, 0.1, 0.15, 0.25, 0.5, 0.75, 1]}

las = ElasticNet(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 84 candidates, totalling 840 fits
Training Accuracy: 0.5675894103664105
Testing Accuracy: 0.520764478468223
{'alpha': 0.001, 'l1_ratio': 0.05}


## Train/Test on Multiple Classifiers

In [30]:
names = ["Linear SVM", "RBF SVM", "Gaussian Process", "Logistic Regression"
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
print(score_dict)

{'Linear SVM': 0.5384615384615384, 'RBF SVM': 0.6461538461538462, 'Gaussian Process': 0.6615384615384615, 'Logistic RegressionDecision Tree': 0.6192307692307693, 'Random Forest': 0.5, 'Neural Net': 0.5384615384615384, 'AdaBoost': 0.5769230769230769, 'Naive Bayes': 0.5384615384615384, 'Bagging Classifier': 0.5884615384615385}


In [22]:
# Vanilla SVM, https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)
print(clf.predict([[-0.8, -1]]))

[1]


In [2]:
# Multinomial Naive Bayes
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
clf = MultinomialNB()
clf.fit(X, y)
print(clf.predict(X[2:3]))

[3]


In [3]:
# Gaussian NB
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = GaussianNB()
clf.fit(X, Y)
print(clf.predict([[-0.8, -1]]))

clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))
print(clf_pf.predict([[-0.8, -1]]))

[1]
[1]


In [4]:
# Logistic Regression, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, max_iter = 50000).fit(X, y)
print(clf.predict(X[:2, :]))

print(clf.predict_proba(X[:2, :]))
print(clf.score(X, y))

[0 0]
[[9.81579028e-01 1.84209573e-02 1.44796627e-08]
 [9.71349907e-01 2.86500630e-02 3.01442199e-08]]
0.9733333333333334


In [32]:
# Linear SVM for large datasets, https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
X, y = datasets.load_digits(n_class=9, return_X_y=True)
data = X / 16
feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=1,
                                n_components=300)
data_transformed = feature_map_nystroem.fit_transform(data)

clf = make_pipeline(StandardScaler(),
                  LinearSVC(random_state=0, tol=1e-5, max_iter = 50000))
clf.fit(data_transformed, y)
clf.score(data_transformed, y)

1.0

In [35]:
# Random Forest Classification, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
print(clf.predict([[0, 0, 0, 0]]))

[1]


In [40]:
# Decision Tree Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
iris = load_iris()
cross_val_score(clf, iris.data, iris.target, cv=10)

array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
       0.86666667, 0.93333333, 1.        , 1.        , 1.        ])

In [38]:
# Bagging Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
X, y = make_classification(n_samples=100, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0).fit(X, y)
clf.predict([[0, 0, 0, 0]])

array([1])

In [43]:
# Gradient Boosting Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
      max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

In [19]:
# MLP Classifier, https://scikit-learn.org/stable/modules/neural_networks_supervised.html
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)
clf.predict([[2., 2.], [-1., -2.]])

array([1, 0])